In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os

from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
from tensorflow.keras.models import load_model
from tensorflow.keras import Model
from tensorflow.keras import Input
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity
from scipy.sparse import csr_matrix

from tqdm.auto import tqdm

import time

In [5]:
articles_df = pd.read_csv("../datasets/articles_transactions_5.csv")
articles_df.shape

(26722, 26)

In [16]:
articles_df = pd.read_csv("../datasets/articles_transactions_5.csv")
t = pd.read_csv("../datasets/transactions_5.csv")
pd.set_option("display.max_rows", None)
i = articles_df.copy()
i['detail_desc'] = i['detail_desc'].fillna("")

In [7]:
vectorizer = TfidfVectorizer(stop_words = "english")
candidate_profile_X = vectorizer.fit_transform(i["detail_desc"])
cosine_sim  = cosine_similarity(candidate_profile_X)
cosine_df = pd.DataFrame(cosine_sim, columns=i['article_id'], index=i['article_id'])
cosine_df.head(25)

article_id,775310002,872813003,872813001,860411004,883068004,866755001,399256002,873428004,873428001,883068001,...,893215001,697050011,783802002,899222001,820944002,570900005,712216011,816995001,682854005,805505001
article_id,,,,,,,,,,,,,,,,,,,,,
775310002,1.000000,0.154155,0.154155,0.318150,0.229418,0.151249,0.096389,0.236930,0.236930,0.229418,...,0.080869,0.000000,0.026519,0.100319,0.086828,0.111945,0.000000,0.000000,0.174451,0.065443
872813003,0.154155,1.000000,1.000000,0.171371,0.119139,0.481732,0.123835,0.063065,0.063065,0.119139,...,0.140385,0.040878,0.090522,0.000000,0.085815,0.080996,0.000000,0.034172,0.028189,0.187386
872813001,0.154155,1.000000,1.000000,0.171371,0.119139,0.481732,0.123835,0.063065,0.063065,0.119139,...,0.140385,0.040878,0.090522,0.000000,0.085815,0.080996,0.000000,0.034172,0.028189,0.187386
860411004,0.318150,0.171371,0.171371,1.000000,0.196065,0.321121,0.181286,0.204027,0.204027,0.196065,...,0.060899,0.000000,0.029481,0.000000,0.043035,0.037254,0.000000,0.000000,0.043255,0.051838
883068004,0.229418,0.119139,0.119139,0.196065,1.000000,0.060451,0.203524,0.459500,0.459500,1.000000,...,0.000000,0.000000,0.049862,0.000000,0.019401,0.000000,0.000000,0.051474,0.000000,0.025102
866755001,0.151249,0.481732,0.481732,0.321121,0.060451,1.000000,0.084335,0.046713,0.046713,0.060451,...,0.096169,0.175858,0.069346,0.000000,0.033870,0.097818,0.000000,0.000000,0.034044,0.298397
399256002,0.096389,0.123835,0.123835,0.181286,0.203524,0.084335,1.000000,0.138661,0.138661,0.203524,...,0.000000,0.000000,0.121014,0.000000,0.016600,0.000000,0.000000,0.000000,0.000000,0.042956
873428004,0.236930,0.063065,0.063065,0.204027,0.459500,0.046713,0.138661,1.000000,1.000000,0.459500,...,0.000000,0.000000,0.000000,0.000000,0.141246,0.043643,0.000000,0.000000,0.075419,0.019397
873428001,0.236930,0.063065,0.063065,0.204027,0.459500,0.046713,0.138661,1.000000,1.000000,0.459500,...,0.000000,0.000000,0.000000,0.000000,0.141246,0.043643,0.000000,0.000000,0.075419,0.019397


In [19]:
def content_based_recommender_Tfid(customer):
    all_similar_items = pd.DataFrame()
    customer_purchases = t['article_id'][t['customer_id'] == customer].drop_duplicates().values
    for item in customer_purchases:
        similar_items = cosine_df[item].sort_values(ascending=False)
        all_similar_items = pd.concat([all_similar_items,similar_items.nlargest(6)])[1:]
    return list(set(all_similar_items[0].nlargest(6).index.values))


In [20]:
pd.set_option("display.max_rows", None)


In [21]:
print(content_based_recommender_Tfid('65cb62c794232651e2ac711faa11c2b4e3d41d5f3b59b50bee3ffde1d5776644'))

[897901001, 897901002, 897901003, 792753001, 792753002, 890866002]


In [22]:
transac = t
u = t['customer_id'].drop_duplicates().to_frame()
i = t['article_id'].drop_duplicates().to_frame()
test_u = u.sample(n=1000, random_state = 1)
test_t = pd.DataFrame()
for k,cust in tqdm(enumerate(test_u['customer_id'])):  
    cust_transac = transac[transac['customer_id'] == cust]
    bottom_transac = cust_transac[-1 * round(0.20 * len(cust_transac)):]
    test_t = test_t.append(bottom_transac)
    indexs = bottom_transac.index
    transac.drop(labels = indexs, axis = 0,inplace=True )
#     print(k)
test_t.shape
test_t.head()

0it [00:00, ?it/s]

C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Loc

C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Loc

C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Loc

C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Loc

...

C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Loc

C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Loc

C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Loc

C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Local\Temp\ipykernel_16656\1968786639.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_t = test_t.append(bottom_transac)
C:\Users\Hammozi\AppData\Loc

,Unnamed: 0.1,Unnamed: 0,t_dat,customer_id,article_id,price,sales_channel_id
117783,1679410,31474231,2020-09-13,8bbf433ee460776600b2750a3a24c911eff36fa25b0397...,723469002,0.022864,2
117784,1679411,31474232,2020-09-13,8bbf433ee460776600b2750a3a24c911eff36fa25b0397...,673677002,0.022864,2
117785,1679412,31474233,2020-09-13,8bbf433ee460776600b2750a3a24c911eff36fa25b0397...,762846027,0.022864,2
511111,1207787,31002608,2020-09-01,559d63db9fc01fbf65b1d53fd3e59d8b6337b311fa3e9b...,910449001,0.033881,2
783799,1022671,30817492,2020-08-27,991502495abd2fb1eb14359997a49fefa52d03b2ab242f...,873678003,0.025407,1


In [23]:
precision_list_tfidf = []
print(test_u.shape)
for j,user in tqdm(enumerate(test_u['customer_id'])):
    rec = content_based_recommender_Tfid(user)
    u_purchases = test_t[test_t['customer_id'] == user]['article_id'].values
    precision_list_tfidf.append(len(np.intersect1d(rec,u_purchases))/5)
print("MEAN Precison of CNN based Recommender",sum(precision_list_tfidf)/len(precision_list_tfidf))

(1000, 1)


0it [00:00, ?it/s]

MEAN Precison of CNN based Recommender 0.020799999999999957
